In [1]:
import json
import logging
import os
import zipfile
from datetime import datetime

from adobe.pdfservices.operation.auth.service_principal_credentials import (
    ServicePrincipalCredentials,
)
from adobe.pdfservices.operation.exception.exceptions import (
    SdkException,
    ServiceApiException,
    ServiceUsageException,
)
from adobe.pdfservices.operation.io.cloud_asset import CloudAsset
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.extract_pdf_job import ExtractPDFJob
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_element_type import (
    ExtractElementType,
)
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_pdf_params import (
    ExtractPDFParams,
)
from adobe.pdfservices.operation.pdfjobs.params.extract_pdf.extract_renditions_element_type import (
    ExtractRenditionsElementType,
)
from adobe.pdfservices.operation.pdfjobs.result.extract_pdf_result import (
    ExtractPDFResult,
)

# Initializeer de logger
logging.basicConfig(level=logging.INFO)


# Dit sample illustreert hoe je tekstinformatie uit een PDF kunt extraheren.
#
# Raadpleeg README.md voor instructies over hoe je de sample uitvoert en de output zip file begrijpt.
class ExtractTextInfoFromPDF:
    def __init__(self) -> None:
        try:
            # Open en lees het input PDF-bestand
            with open("./FM_Sec12.pdf", "rb") as file:
                input_stream = file.read()
            # Lees de credentials uit het JSON bestand
            creds = self.load_pdf_service_credentials()
            credentials = ServicePrincipalCredentials(
                client_id=creds["client_id"], client_secret=creds["client_secret"]
            )
            # Maak een instantie van de PDF Services
            pdf_services = PDFServices(credentials=credentials)
            # Upload het bronbestand als een asset
            input_asset = pdf_services.upload(input_stream=input_stream, mime_type=PDFServicesMediaType.PDF)
            # Stel de parameters in voor de extractie, hierbij enkel TEXT elementen extraheren
            extract_pdf_params = ExtractPDFParams(
                elements_to_extract=[
                    ExtractElementType.TEXT,
                    ExtractElementType.TABLES,
                ],
                elements_to_extract_renditions=[ExtractRenditionsElementType.FIGURES],
            )
            # Maak en verstuur de extractiejob
            extract_pdf_job = ExtractPDFJob(input_asset=input_asset, extract_pdf_params=extract_pdf_params)
            location = pdf_services.submit(extract_pdf_job)
            pdf_services_response = pdf_services.get_job_result(location, ExtractPDFResult)
            # Verkrijg de resulterende asset en de bijbehorende content
            result_asset: CloudAsset = pdf_services_response.get_result().get_resource()
            stream_asset: StreamAsset = pdf_services.get_content(result_asset)
            # Schrijf de output naar een zipbestand
            output_file_path = self.create_output_file_path()
            with open(output_file_path, "wb") as file:
                file.write(stream_asset.get_input_stream())
            # Open het zipbestand en lees de JSON data uit 'structuredData.json'
            archive = zipfile.ZipFile(output_file_path, "r")
            with archive.open("structuredData.json") as jsonentry:
                jsondata = jsonentry.read()
            data = json.loads(jsondata)
            # Loop door de elementen en print de tekst als het pad eindigt op '/H1'
            for element in data["elements"]:
                if element["Path"].endswith("/H1"):
                    print(element["Text"])

        except (ServiceApiException, ServiceUsageException, SdkException) as e:
            logging.exception(f"Er is een fout opgetreden tijdens de uitvoering: {e}")

    # Deze static method maakt een outputbestandspad op basis van de huidige datum en tijd
    @staticmethod
    def create_output_file_path() -> str:
        now = datetime.now()
        time_stamp = now.strftime("%Y-%m-%dT%H-%M-%S")
        os.makedirs("output/ExtractTextInfoFromPDF", exist_ok=True)
        return f"output/ExtractTextInfoFromPDF/extract{time_stamp}.zip"

    def load_pdf_service_credentials(self) -> dict:
        """
        Leest de credentials uit het JSON bestand en retourneert een dictionary.
        """
        with open("pdfservices-api-credentials.json") as cred_file:
            data = json.load(cred_file)
        return data["client_credentials"]


if __name__ == "__main__":
    ExtractTextInfoFromPDF()

FileNotFoundError: [Errno 2] No such file or directory: './FM_Sec12.pdf'